In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 40,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [35, 36, 37, 38, 39, 40]
)


In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=42'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  for col_index in [2, 3]:
    cols = [data.text.strip() for data in row.find_all('td')]
    if (cols[col_index] == ''):
      continue

    # cols = [data.text.strip() for data in row.find_all('td')]
    
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_b2)
    cols.append(post_str_b3)
    cols.append(post_str_b4)
    cols.append(post_str_b5)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_b2 = str_combo[1]
    post_str_b3 = str_combo[2]
    post_str_b4 = str_combo[3]
    post_str_b5 = str_combo[4]
    post_str_blast = str_combo[-1]



# lst = []
# for row in rows:
  
  
#   for col_index in [2, 3]:
#     cols = [data.text.strip() for data in row.find_all('td')]
#     if (cols[col_index] == ''):
#       continue
  
#     id, bolillas_int = formalize_data(bolillas_str=cols[col_index])
#     cols.append(id)
#     cols.append(bolillas_int)
#     cols.extend([b for b in bolillas_int])
    
#     is_rare, reason = rr.is_rare_combination(bolillas_int, id)
#     cols.append(is_rare)
#     cols.append(reason)
#     left_scores = [round(b, 4) for b in rr.calculate_left_scores(bolillas_int)]

#     right_scores = [round(b, 4) for b in rr.calculate_right_scores(bolillas_int)]
#     lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
#     row_score = round(sum(lr_scores),4)

#     shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
#     shared_values = [round(s, 4) for s in shared_values]
#     cols.extend([score for score in left_scores])
#     cols.extend([score for score in right_scores])
#     # cols.extend(lr_scores)
#     cols.append(shared_values)
#     cols.append(len(shared_values))
#     cols.append(row_score)
#     cols.append(rr.jumps_map(bolillas_int))
#     cols.append(rr.unijump(bolillas_int))
    
#     lst.append(cols)

with open('./kbl_cache_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=5350


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""",null,null,null,null,null,null,true,16,"""02_05_21_08_01""",77
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""05/07/2025""","""1830""","""30 18 09 20 26 10""","""33 23 16 25 03 05""","""""","""""","""091018202630""","[9, 10, … 30]",9,10,18,20,26,30,"""09""","""10""","""18""","""20""","""26""","""30""","""02""","""04""","""09""","""30""","""38""","""39""",false,0,"""01_08_02_06_04""",-39
"""05/07/2025""","""1830""","""30 18 09 20 26 10""","""33 23 16 25 03 05""","""""","""""","""091018202630""","[9, 10, … 30]",9,10,18,20,26,30,"""09""","""10""","""18""","""20""","""26""","""30""","""09""","""10""","""18""","""20""","""26""","""30""",false,0,"""01_08_02_06_04""",-39
"""03/07/2025""","""1829""","""09 03 20 35 15 27""","""27 14 06 13 34 08""","""""","""""","""030915202735""","[3, 9, … 35]",3,9,15,20,27,35,"""03""","""09""","""15""","""20""","""27""","""35""","""09""","""10""","""18""","""20""","""26""","""30""",true,8,"""06_06_05_07_08""",-8
"""03/07/2025""","""1829""","""09 03 20 35 15 27""","""27 14 06 13 34 08""","""""","""""","""030915202735""","[3, 9, … 35]",3,9,15,20,27,35,"""03""","""09""","""15""","""20""","""27""","""35""","""03""","""09""","""15""","""20""","""27""","""35""",true,8,"""06_06_05_07_08""",-8
"""01/07/2025""","""1828""","""16 38 01 26 33 34""","""23 16 02 27 28 14""","""""","""""","""011626333438""","[1, 16, … 38]",1,16,26,33,34,38,"""01""","""16""","""26""","""33""","""34""","""38""","""03""","""09""","""15""","""20""","""27""","""35""",false,0,"""15_10_07_01_04""",17
"""01/07/2025""","""1828""","""16 38 01 26 33 34""","""23 16 02 27 28 14""","""""","""""","""011626333438""","[1, 16, … 38]",1,16,26,33,34,38,"""01""","""16""","""26""","""33""","""34""","""38""","""01""","""16""","""26""","""33""","""34""","""38""",false,0,"""15_10_07_01_04""",17
"""28/06/2025""","""1827""","""04 16 21 30 19 32""","""16 27 09 21 39 37""","""""","""""","""041619213032""","[4, 16, … 32]",4,16,19,21,30,32,"""04""","""16""","""19""","""21""","""30""","""32""","""01""","""16""","""26""","""33""","""34""","""38""",false,0,"""12_03_02_09_02""",-22


In [5]:
dfRows.item(0, 'b2')

4

In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [7]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='02'
df_viz FOR str_b1 = 02


post_str_b1,count,percent
str,u32,f64
"""02""",285,41.364296
"""01""",74,10.740203
"""03""",70,10.159652
"""04""",44,6.386067
"""05""",31,4.499274
…,…,…
"""19""",4,0.580552
"""16""",3,0.435414
"""20""",2,0.290276


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 02
Last 3 results last_results=['02', '09']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""24/06/2025""","""1825""","""16 18 24 26 39 02""","""32 31 23 18 12 33""","""""","""""","""021618242639""","[2, 16, … 39]",2,16,18,24,26,39,"""02""","""16""","""18""","""24""","""26""","""39""","""09""","""21""","""25""","""28""","""35""","""37""",false,0,"""14_02_06_02_13""",47
"""24/06/2025""","""1825""","""16 18 24 26 39 02""","""32 31 23 18 12 33""","""""","""""","""021618242639""","[2, 16, … 39]",2,16,18,24,26,39,"""02""","""16""","""18""","""24""","""26""","""39""","""02""","""16""","""18""","""24""","""26""","""39""",false,0,"""14_02_06_02_13""",47
"""21/06/2025""","""1824""","""09 30 04 08 13 02""","""03 27 12 16 01 23""","""""","""""","""020408091330""","[2, 4, … 30]",2,4,8,9,13,30,"""02""","""04""","""08""","""09""","""13""","""30""","""02""","""16""","""18""","""24""","""26""","""39""",false,0,"""02_04_01_04_17""",-7
"""21/06/2025""","""1824""","""09 30 04 08 13 02""","""03 27 12 16 01 23""","""""","""""","""020408091330""","[2, 4, … 30]",2,4,8,9,13,30,"""02""","""04""","""08""","""09""","""13""","""30""","""02""","""04""","""08""","""09""","""13""","""30""",false,0,"""02_04_01_04_17""",-7
"""12/06/2025""","""1820""","""37 13 40 23 12 02""","""20 17 10 38 09 21""","""""","""""","""021213233740""","[2, 12, … 40]",2,12,13,23,37,40,"""02""","""12""","""13""","""23""","""37""","""40""","""03""","""06""","""17""","""21""","""28""","""32""",true,1,"""10_01_10_14_03""",13
"""12/06/2025""","""1820""","""37 13 40 23 12 02""","""20 17 10 38 09 21""","""""","""""","""021213233740""","[2, 12, … 40]",2,12,13,23,37,40,"""02""","""12""","""13""","""23""","""37""","""40""","""02""","""12""","""13""","""23""","""37""","""40""",true,1,"""10_01_10_14_03""",13
"""27/05/2025""","""1813""","""19 02 23 39 08 03""","""03 25 18 39 33 20""","""""","""""","""020308192339""","[2, 3, … 39]",2,3,8,19,23,39,"""02""","""03""","""08""","""19""","""23""","""39""","""08""","""11""","""25""","""31""","""36""","""38""",true,1,"""01_05_11_04_16""",47
"""27/05/2025""","""1813""","""19 02 23 39 08 03""","""03 25 18 39 33 20""","""""","""""","""020308192339""","[2, 3, … 39]",2,3,8,19,23,39,"""02""","""03""","""08""","""19""","""23""","""39""","""02""","""03""","""08""","""19""","""23""","""39""",true,1,"""01_05_11_04_16""",47


df_jumps_map FOR str_b1 = 02


jumps_map,count
str,u32
"""01_02_03_15_06""",2
"""03_07_06_11_10""",2
"""11_09_02_07_02""",2
"""04_03_08_02_08""",2
"""04_15_02_09_05""",2
…,…
"""02_08_07_06_07""",1
"""03_03_03_03_26""",1
"""09_06_02_04_16""",1


df_unijump FOR str_b1 = 02


unijump,count
i64,u32
3,14
47,13
0,12
-17,12
-37,11
…,…
45,1
49,1
121,1


In [8]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='04'
df_viz FOR str_b2 = 04


post_str_b2,count,percent
str,u32,f64
"""04""",85,33.464567
"""07""",17,6.692913
"""08""",15,5.905512
"""11""",14,5.511811
"""05""",12,4.724409
…,…,…
"""18""",3,1.181102
"""23""",2,0.787402
"""22""",1,0.393701


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 04
Last 3 results last_results=['04', '16']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""21/06/2025""","""1824""","""09 30 04 08 13 02""","""03 27 12 16 01 23""","""""","""""","""020408091330""","[2, 4, … 30]",2,4,8,9,13,30,"""02""","""04""","""08""","""09""","""13""","""30""","""02""","""16""","""18""","""24""","""26""","""39""",false,0,"""02_04_01_04_17""",-7
"""21/06/2025""","""1824""","""09 30 04 08 13 02""","""03 27 12 16 01 23""","""""","""""","""020408091330""","[2, 4, … 30]",2,4,8,9,13,30,"""02""","""04""","""08""","""09""","""13""","""30""","""02""","""04""","""08""","""09""","""13""","""30""",false,0,"""02_04_01_04_17""",-7
"""17/06/2025""","""1822""","""08 13 04 19 03 18""","""15 18 05 02 22 20""","""""","""""","""030408131819""","[3, 4, … 19]",3,4,8,13,18,19,"""03""","""04""","""08""","""13""","""18""","""19""","""04""","""11""","""18""","""20""","""29""","""36""",true,24,"""01_04_05_05_01""",-4
"""17/06/2025""","""1822""","""08 13 04 19 03 18""","""15 18 05 02 22 20""","""""","""""","""030408131819""","[3, 4, … 19]",3,4,8,13,18,19,"""03""","""04""","""08""","""13""","""18""","""19""","""03""","""04""","""08""","""13""","""18""","""19""",true,24,"""01_04_05_05_01""",-4
"""01/05/2025""","""1802""","""21 34 01 32 04 24""","""07 39 16 08 17 26""","""""","""""","""010421243234""","[1, 4, … 34]",1,4,21,24,32,34,"""01""","""04""","""21""","""24""","""32""","""34""","""05""","""11""","""12""","""22""","""23""","""35""",false,0,"""03_17_03_08_02""",-77
"""01/05/2025""","""1802""","""21 34 01 32 04 24""","""07 39 16 08 17 26""","""""","""""","""010421243234""","[1, 4, … 34]",1,4,21,24,32,34,"""01""","""04""","""21""","""24""","""32""","""34""","""01""","""04""","""21""","""24""","""32""","""34""",false,0,"""03_17_03_08_02""",-77
"""03/08/2024""","""1686""","""02 04 08 26 05 33""","""30 12 22 29 38 23""","""""","""""","""020405082633""","[2, 4, … 33]",2,4,5,8,26,33,"""02""","""04""","""05""","""08""","""26""","""33""","""01""","""02""","""09""","""16""","""22""","""31""",true,16,"""02_01_03_18_07""",-49
"""03/08/2024""","""1686""","""02 04 08 26 05 33""","""30 12 22 29 38 23""","""""","""""","""020405082633""","[2, 4, … 33]",2,4,5,8,26,33,"""02""","""04""","""05""","""08""","""26""","""33""","""02""","""04""","""05""","""08""","""26""","""33""",true,16,"""02_01_03_18_07""",-49


df_jumps_map FOR str_b2 = 04


jumps_map,count
str,u32
"""01_16_03_05_09""",2
"""02_01_03_18_07""",2
"""03_05_01_15_14""",2
"""01_04_03_10_10""",2
"""02_03_01_16_11""",2
…,…
"""02_02_09_16_03""",1
"""01_16_03_08_07""",1
"""01_02_01_13_06""",1


df_unijump FOR str_b2 = 04


unijump,count
i64,u32
-49,10
2,6
-37,5
0,5
-39,5
…,…
62,1
-96,1
-5,1


In [9]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='09'
df_viz FOR str_b3 = 09


post_str_b3,count,percent
str,u32,f64
"""09""",59,34.911243
"""16""",9,5.325444
"""13""",8,4.733728
"""15""",8,4.733728
"""21""",7,4.142012
…,…,…
"""28""",2,1.183432
"""34""",1,0.591716
"""32""",1,0.591716


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 09
Last 3 results last_results=['09', '15']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""20/02/2025""","""1772""","""09 32 21 05 03 27""","""09 20 15 01 07 30""","""""","""""","""030509212732""","[3, 5, … 32]",3,5,9,21,27,32,"""03""","""05""","""09""","""21""","""27""","""32""","""01""","""07""","""15""","""18""","""25""","""26""",false,0,"""02_04_12_06_05""",39
"""20/02/2025""","""1772""","""09 32 21 05 03 27""","""09 20 15 01 07 30""","""""","""""","""030509212732""","[3, 5, … 32]",3,5,9,21,27,32,"""03""","""05""","""09""","""21""","""27""","""32""","""03""","""05""","""09""","""21""","""27""","""32""",false,0,"""02_04_12_06_05""",39
"""30/11/2024""","""1737""","""10 09 05 37 03 39""","""14 10 15 02 11 34""","""""","""""","""030509103739""","[3, 5, … 39]",3,5,9,10,37,39,"""03""","""05""","""09""","""10""","""37""","""39""","""09""","""10""","""14""","""28""","""35""","""36""",true,16,"""02_04_01_27_02""",-114
"""30/11/2024""","""1737""","""10 09 05 37 03 39""","""14 10 15 02 11 34""","""""","""""","""030509103739""","[3, 5, … 39]",3,5,9,10,37,39,"""03""","""05""","""09""","""10""","""37""","""39""","""03""","""05""","""09""","""10""","""37""","""39""",true,16,"""02_04_01_27_02""",-114
"""19/11/2024""","""1732""","""01 23 19 40 09 02""","""27 40 24 08 16 18""","""""","""""","""010209192340""","[1, 2, … 40]",1,2,9,19,23,40,"""01""","""02""","""09""","""19""","""23""","""40""","""03""","""12""","""14""","""16""","""26""","""31""",false,0,"""01_07_10_04_17""",34
"""19/11/2024""","""1732""","""01 23 19 40 09 02""","""27 40 24 08 16 18""","""""","""""","""010209192340""","[1, 2, … 40]",1,2,9,19,23,40,"""01""","""02""","""09""","""19""","""23""","""40""","""01""","""02""","""09""","""19""","""23""","""40""",false,0,"""01_07_10_04_17""",34
"""26/09/2024""","""1709""","""27 39 02 09 07 25""","""25 33 09 07 14 34""","""""","""""","""020709252739""","[2, 7, … 39]",2,7,9,25,27,39,"""02""","""07""","""09""","""25""","""27""","""39""","""14""","""17""","""25""","""29""","""34""","""37""",true,16,"""05_02_16_02_12""",97
"""26/09/2024""","""1709""","""27 39 02 09 07 25""","""25 33 09 07 14 34""","""""","""""","""020709252739""","[2, 7, … 39]",2,7,9,25,27,39,"""02""","""07""","""09""","""25""","""27""","""39""","""02""","""07""","""09""","""25""","""27""","""39""",true,16,"""05_02_16_02_12""",97


df_jumps_map FOR str_b3 = 09


jumps_map,count
str,u32
"""01_06_02_08_09""",2
"""02_02_01_14_12""",2
"""06_02_16_02_07""",2
"""02_04_06_03_08""",2
"""04_01_12_06_01""",2
…,…
"""01_05_02_09_12""",1
"""01_03_05_04_16""",1
"""06_01_15_03_08""",1


df_unijump FOR str_b3 = 09


unijump,count
i64,u32
51,7
23,5
34,5
-34,5
0,5
…,…
-10,1
1,1
10,1


In [10]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='30'
df_viz FOR str_b4 = 30


post_str_b4,count,percent
str,u32,f64
"""30""",81,34.322034
"""20""",12,5.084746
"""23""",10,4.237288
"""26""",10,4.237288
"""24""",9,3.813559
…,…,…
"""10""",2,0.847458
"""08""",2,0.847458
"""35""",2,0.847458


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 30
Last 3 results last_results=['30', '20']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""24/04/2025""","""1799""","""16 14 17 34 30 40""","""33 17 18 39 03 09""","""""","""""","""141617303440""","[14, 16, … 40]",14,16,17,30,34,40,"""14""","""16""","""17""","""30""","""34""","""40""","""03""","""09""","""19""","""20""","""29""","""35""",true,16,"""02_01_13_04_06""",66
"""24/04/2025""","""1799""","""16 14 17 34 30 40""","""33 17 18 39 03 09""","""""","""""","""141617303440""","[14, 16, … 40]",14,16,17,30,34,40,"""14""","""16""","""17""","""30""","""34""","""40""","""14""","""16""","""17""","""30""","""34""","""40""",true,16,"""02_01_13_04_06""",66
"""17/09/2024""","""1705""","""34 37 10 17 03 30""","""03 19 07 13 35 02""","""""","""""","""031017303437""","[3, 10, … 37]",3,10,17,30,34,37,"""03""","""10""","""17""","""30""","""34""","""37""","""11""","""16""","""17""","""20""","""34""","""35""",true,24,"""07_07_13_04_03""",44
"""17/09/2024""","""1705""","""34 37 10 17 03 30""","""03 19 07 13 35 02""","""""","""""","""031017303437""","[3, 10, … 37]",3,10,17,30,34,37,"""03""","""10""","""17""","""30""","""34""","""37""","""03""","""10""","""17""","""30""","""34""","""37""",true,24,"""07_07_13_04_03""",44
"""10/09/2024""","""1702""","""15 34 02 30 33 09""","""40 18 07 19 28 12""","""""","""""","""020915303334""","[2, 9, … 34]",2,9,15,30,33,34,"""02""","""09""","""15""","""30""","""33""","""34""","""04""","""19""","""21""","""26""","""34""","""40""",false,0,"""07_06_15_03_01""",62
"""10/09/2024""","""1702""","""15 34 02 30 33 09""","""40 18 07 19 28 12""","""""","""""","""020915303334""","[2, 9, … 34]",2,9,15,30,33,34,"""02""","""09""","""15""","""30""","""33""","""34""","""02""","""09""","""15""","""30""","""33""","""34""",false,0,"""07_06_15_03_01""",62
"""09/07/2024""","""1675""","""39 30 14 17 26 37""","""05 14 20 36 16 35""","""""","""""","""141726303739""","[14, 17, … 39]",14,17,26,30,37,39,"""14""","""17""","""26""","""30""","""37""","""39""","""05""","""17""","""19""","""31""","""33""","""37""",false,0,"""03_09_04_07_02""",-35
"""09/07/2024""","""1675""","""39 30 14 17 26 37""","""05 14 20 36 16 35""","""""","""""","""141726303739""","[14, 17, … 39]",14,17,26,30,37,39,"""14""","""17""","""26""","""30""","""37""","""39""","""14""","""17""","""26""","""30""","""37""","""39""",false,0,"""03_09_04_07_02""",-35


df_jumps_map FOR str_b4 = 30


jumps_map,count
str,u32
"""11_16_01_06_04""",2
"""05_20_04_02_06""",2
"""03_09_05_03_06""",2
"""02_01_13_04_06""",2
"""12_08_02_03_03""",2
…,…
"""02_14_10_02_02""",1
"""20_03_06_03_06""",1
"""10_04_12_06_02""",1


df_unijump FOR str_b4 = 30


unijump,count
i64,u32
37,7
-35,5
83,5
11,5
6,5
…,…
13,1
31,1
46,1


In [11]:
possible_b5, exclude_jump_maps_b5, last_post_b5 = plot_post_col(col='str_b5', post_col='post_str_b5')

Setting test_value='38'
df_viz FOR str_b5 = 38


post_str_b5,count,percent
str,u32,f64
"""38""",59,33.522727
"""34""",12,6.818182
"""32""",9,5.113636
"""35""",9,5.113636
"""30""",8,4.545455
…,…,…
"""17""",1,0.568182
"""19""",1,0.568182
"""16""",1,0.568182


:Overlay
   .Bars.I    :Bars   [post_str_b5]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 38
Last 3 results last_results=['38', '26']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""01/03/2025""","""1776""","""02 34 38 14 22 40""","""09 08 36 39 14 01""","""""","""""","""021422343840""","[2, 14, … 40]",2,14,22,34,38,40,"""02""","""14""","""22""","""34""","""38""","""40""","""04""","""06""","""11""","""18""","""26""","""30""",true,4,"""12_08_12_04_02""",38
"""01/03/2025""","""1776""","""02 34 38 14 22 40""","""09 08 36 39 14 01""","""""","""""","""021422343840""","[2, 14, … 40]",2,14,22,34,38,40,"""02""","""14""","""22""","""34""","""38""","""40""","""02""","""14""","""22""","""34""","""38""","""40""",true,4,"""12_08_12_04_02""",38
"""15/10/2024""","""1717""","""38 18 19 39 23 37""","""20 31 34 02 07 21""","""""","""""","""181923373839""","[18, 19, … 39]",18,19,23,37,38,39,"""18""","""19""","""23""","""37""","""38""","""39""","""11""","""14""","""19""","""21""","""30""","""36""",true,8,"""01_04_14_01_01""",66
"""15/10/2024""","""1717""","""38 18 19 39 23 37""","""20 31 34 02 07 21""","""""","""""","""181923373839""","[18, 19, … 39]",18,19,23,37,38,39,"""18""","""19""","""23""","""37""","""38""","""39""","""18""","""19""","""23""","""37""","""38""","""39""",true,8,"""01_04_14_01_01""",66
"""20/07/2024""","""1680""","""22 39 20 38 19 09""","""05 04 11 15 25 21""","""""","""""","""091920223839""","[9, 19, … 39]",9,19,20,22,38,39,"""09""","""19""","""20""","""22""","""38""","""39""","""08""","""10""","""21""","""22""","""23""","""39""",false,0,"""10_01_02_16_01""",-45
"""20/07/2024""","""1680""","""22 39 20 38 19 09""","""05 04 11 15 25 21""","""""","""""","""091920223839""","[9, 19, … 39]",9,19,20,22,38,39,"""09""","""19""","""20""","""22""","""38""","""39""","""09""","""19""","""20""","""22""","""38""","""39""",false,0,"""10_01_02_16_01""",-45
"""30/05/2024""","""1658""","""36 10 38 39 15 27""","""34 32 36 19 09 17""","""""","""""","""101527363839""","[10, 15, … 39]",10,15,27,36,38,39,"""10""","""15""","""27""","""36""","""38""","""39""","""05""","""17""","""22""","""35""","""37""","""39""",false,0,"""05_12_09_02_01""",4
"""30/05/2024""","""1658""","""36 10 38 39 15 27""","""34 32 36 19 09 17""","""""","""""","""101527363839""","[10, 15, … 39]",10,15,27,36,38,39,"""10""","""15""","""27""","""36""","""38""","""39""","""10""","""15""","""27""","""36""","""38""","""39""",false,0,"""05_12_09_02_01""",4


df_jumps_map FOR str_b5 = 38


jumps_map,count
str,u32
"""14_01_12_05_02""",2
"""09_02_18_02_02""",2
"""07_12_15_03_01""",2
"""05_14_04_07_01""",2
"""22_01_04_10_02""",2
…,…
"""05_05_21_02_02""",1
"""13_03_04_12_01""",1
"""08_01_14_01_02""",1


df_unijump FOR str_b5 = 38


unijump,count
i64,u32
-46,6
4,5
38,5
-22,5
-45,4
…,…
106,1
-27,1
62,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='39'
df_viz FOR str_b6 = 39


post_str_blast,count,percent
str,u32,f64
"""39""",298,42.389758
"""38""",63,8.961593
"""40""",55,7.823613
"""37""",50,7.112376
"""35""",33,4.694168
…,…,…
"""19""",2,0.284495
"""14""",1,0.142248
"""23""",1,0.142248


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 39
Last 3 results last_results=['37', '39']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/07/2025""","""1831""","""39 30 04 09 38 02""","""10 11 06 28 04 40""","""""","""""","""020409303839""","[2, 4, … 39]",2,4,9,30,38,39,"""02""","""04""","""09""","""30""","""38""","""39""","""02""","""04""","""09""","""30""","""38""","""39""",true,16,"""02_05_21_08_01""",77
"""24/06/2025""","""1825""","""16 18 24 26 39 02""","""32 31 23 18 12 33""","""""","""""","""021618242639""","[2, 16, … 39]",2,16,18,24,26,39,"""02""","""16""","""18""","""24""","""26""","""39""","""09""","""21""","""25""","""28""","""35""","""37""",false,0,"""14_02_06_02_13""",47
"""24/06/2025""","""1825""","""16 18 24 26 39 02""","""32 31 23 18 12 33""","""""","""""","""021618242639""","[2, 16, … 39]",2,16,18,24,26,39,"""02""","""16""","""18""","""24""","""26""","""39""","""02""","""16""","""18""","""24""","""26""","""39""",false,0,"""14_02_06_02_13""",47
"""27/05/2025""","""1813""","""19 02 23 39 08 03""","""03 25 18 39 33 20""","""""","""""","""020308192339""","[2, 3, … 39]",2,3,8,19,23,39,"""02""","""03""","""08""","""19""","""23""","""39""","""08""","""11""","""25""","""31""","""36""","""38""",true,1,"""01_05_11_04_16""",47
"""27/05/2025""","""1813""","""19 02 23 39 08 03""","""03 25 18 39 33 20""","""""","""""","""020308192339""","[2, 3, … 39]",2,3,8,19,23,39,"""02""","""03""","""08""","""19""","""23""","""39""","""02""","""03""","""08""","""19""","""23""","""39""",true,1,"""01_05_11_04_16""",47
"""05/04/2025""","""1791""","""22 18 39 13 23 30""","""19 09 10 17 22 07""","""""","""""","""131822233039""","[13, 18, … 39]",13,18,22,23,30,39,"""13""","""18""","""22""","""23""","""30""","""39""","""04""","""17""","""28""","""38""","""39""","""40""",false,0,"""05_04_01_07_09""",-24
"""05/04/2025""","""1791""","""22 18 39 13 23 30""","""19 09 10 17 22 07""","""""","""""","""131822233039""","[13, 18, … 39]",13,18,22,23,30,39,"""13""","""18""","""22""","""23""","""30""","""39""","""13""","""18""","""22""","""23""","""30""","""39""",false,0,"""05_04_01_07_09""",-24
"""03/04/2025""","""1790""","""16 17 32 34 39 13""","""26 31 36 34 28 08""","""""","""""","""131617323439""","[13, 16, … 39]",13,16,17,32,34,39,"""13""","""16""","""17""","""32""","""34""","""39""","""13""","""18""","""22""","""23""","""30""","""39""",false,0,"""03_01_15_02_05""",86
"""03/04/2025""","""1790""","""16 17 32 34 39 13""","""26 31 36 34 28 08""","""""","""""","""131617323439""","[13, 16, … 39]",13,16,17,32,34,39,"""13""","""16""","""17""","""32""","""34""","""39""","""13""","""16""","""17""","""32""","""34""","""39""",false,0,"""03_01_15_02_05""",86


df_jumps_map FOR str_b6 = 39


jumps_map,count
str,u32
"""05_18_02_04_09""",2
"""09_11_01_13_02""",2
"""03_15_04_10_04""",2
"""01_04_14_01_01""",2
"""10_03_01_15_06""",2
…,…
"""18_02_06_04_06""",1
"""08_18_04_01_02""",1
"""08_04_01_01_15""",1


df_unijump FOR str_b6 = 39


unijump,count
i64,u32
-22,16
47,11
16,11
-15,11
31,10
…,…
-80,1
-112,1
101,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_kbl_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []

  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)


  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  
  # dfRows = pl.DataFrame(
  #   lst,
  #   schema=[
  #     'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
  #     'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
  #     'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
  #     'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
  #     'is_rare_combination', 'reason',
  #     "jumps_map", "unijump"
  #   ],
  #   schema_overrides={
  #     'combo': pl.Array(pl.UInt8, 6),
  #     'b1': pl.UInt8,
  #     'b2': pl.UInt8,
  #     'b3': pl.UInt8,
  #     'b4': pl.UInt8,
  #     'b5': pl.UInt8,
  #     'b6': pl.UInt8,
  #     'reason': pl.UInt8
  #   },
  #   orient='row'
  # )

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [15]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_b5),
  pl.col("str_b6").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_b5),
  ~pl.col("str_b6").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""010513232433""","[1, 5, … 33]",1,5,13,23,24,33,"""01""","""05""","""13""","""23""","""24""","""33""",false,0,"""04_08_10_01_09""",37
"""010513232434""","[1, 5, … 34]",1,5,13,23,24,34,"""01""","""05""","""13""","""23""","""24""","""34""",false,0,"""04_08_10_01_10""",38
"""010513232435""","[1, 5, … 35]",1,5,13,23,24,35,"""01""","""05""","""13""","""23""","""24""","""35""",false,0,"""04_08_10_01_11""",39
"""010513232436""","[1, 5, … 36]",1,5,13,23,24,36,"""01""","""05""","""13""","""23""","""24""","""36""",false,0,"""04_08_10_01_12""",40
"""010513232438""","[1, 5, … 38]",1,5,13,23,24,38,"""01""","""05""","""13""","""23""","""24""","""38""",false,0,"""04_08_10_01_14""",42
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""041121283435""","[4, 11, … 35]",4,11,21,28,34,35,"""04""","""11""","""21""","""28""","""34""","""35""",false,0,"""07_10_07_06_01""",-14
"""041121283436""","[4, 11, … 36]",4,11,21,28,34,36,"""04""","""11""","""21""","""28""","""34""","""36""",false,0,"""07_10_07_06_02""",-13
"""041121283438""","[4, 11, … 38]",4,11,21,28,34,38,"""04""","""11""","""21""","""28""","""34""","""38""",false,0,"""07_10_07_06_04""",-11


In [16]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""030521283034""","[3, 5, … 34]",3,5,21,28,30,34,"""03""","""05""","""21""","""28""","""30""","""34""",false,0,"""02_16_07_02_04""",-24
